In [1]:
!pip install transformers datasets evaluate ijson emoji;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 29.8 MB/s 
     |████████████████████████████████| 451 kB 67.1 MB/s 
     |████████████████████████████████| 72 kB 487 kB/s 
     |████████████████████████████████| 133 kB 61.9 MB/s 
     |████████████████████████████████| 7.6 MB 82.4 MB/s 
     |████████████████████████████████| 182 kB 69.7 MB/s 
     |████████████████████████████████| 212 kB 101.3 MB/s 
     |████████████████████████████████| 132 kB 102.8 MB/s 
     |████████████████████████████████| 127 kB 84.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import ijson
import os
import logging
import pandas as pd

import torch
import torch.nn as nn
import evaluate

from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [4]:
# def get_dataset(path_to_data):
#   with open(path_to_data) as f:
#     data = json.loads(f.read())
#   data = [[' </s> '.join(x['tweets']), 1 if x['label']=='bot' else 0] for x in data]
#   df = pd.DataFrame(data)
#   df.columns = ["text", "label"]
#   data = Dataset.from_pandas(df)
#   return data


def get_dataset(path_to_data):

  data = []

  with open(path_to_data, 'r') as f:
      objects = ijson.items(f, 'item')
      x = next(objects, 0)
      while x!=0:
        data.append([x['description'], 1 if x['label']=='bot' else 0])
        x = next(objects, 0)
  df = pd.DataFrame(data)
  df.columns = ["text", "label"]
  data = Dataset.from_pandas(df)
  return data


In [5]:
data_args = {'path_to_train' : '/content/drive/MyDrive/advNLP/twibot22/processed/processed_train.json',
             'path_to_test' : '/content/drive/MyDrive/advNLP/twibot22/processed/processed_test.json',
             'path_to_val' : '/content/drive/MyDrive/advNLP/twibot22/processed/processed_val.json'}

max_length = 128
checkpoint = "vinai/bertweet-base"

In [6]:
train_data = get_dataset(data_args['path_to_train'])
val_data = get_dataset(data_args['path_to_val'])
test_data = get_dataset(data_args['path_to_test'])

data = DatasetDict({
    'train': train_data,
    'test': test_data,
    'valid': val_data})

In [10]:
! pip3 install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 6.7 MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49735 sha256=85d309c92c7b2eae9731fb83475420e856b76e465c51ed2bebaf64b6af10b11b
  Stored in directory: /root/.cache/pip/wheels/43/3d/82/e7baffa5e86346c6178d7750dba6e8ef063282a37fc563f8f8
Successfully built emoji


In [11]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True, padding=True, max_length=max_length)

tokenized_dataset = data.map(preprocess_function, batched=True)

  0%|          | 0/700 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

In [12]:
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label = {0: 0, 1: 1}
label2id = {0: 0, 1: 1}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, 
                                                           num_labels=2, 
                                                           id2label=id2label, 
                                                           label2id=label2id)


freeze_pretrained_layers = True

if freeze_pretrained_layers:
  for param in model.base_model.parameters():
      param.requires_grad = False

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [19]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/advNLP/outputs/tweets/bertweet",
    learning_rate=4e-5,
    per_device_train_batch_size=1024,
    per_device_eval_batch_size=1024,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 700000
  Num Epochs = 10
  Instantaneous batch size per device = 1024
  Total train batch size (w. parallel, distributed & accumulation) = 1024
  Gradient Accumulation steps = 1
  Total optimization steps = 6840
  Number of trainable parameters = 592130


Epoch,Training Loss,Validation Loss


In [ ]:
from evaluate import evaluator

task_evaluator = evaluator("text-classification")

test_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data=tokenized_dataset['test'],
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={0: 0, 1: 1}
)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [ ]:
test_results

{'accuracy': 0.6542688081149619,
 'recall': 0.859375,
 'precision': 0.6329113924050633,
 'f1': 0.728959575878065,
 'total_time_in_seconds': 62.13790452600006,
 'samples_per_second': 19.038298909886848,
 'latency_in_seconds': 0.05252570120541002}

In [ ]:
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data=tokenized_dataset['valid'],
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={0: 0, 1: 1}
)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [ ]:
eval_results

{'accuracy': 0.678646934460888,
 'recall': 0.8894858019953953,
 'precision': 0.6529577464788733,
 'f1': 0.7530864197530863,
 'total_time_in_seconds': 109.66953663699996,
 'samples_per_second': 21.564785194889787,
 'latency_in_seconds': 0.04637189709809724}